# Answering Business Questions using SQL

The Chinook database is provided as a SQLite database file called chinook.db

## Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

It's always possible to query the database to get a list of all tables and views in the database in the following way:

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - I have the artist names, and the genre of music they produce:

- Regal: Hip-Hop
- Red Tone: Punk
- Meteor and the Girls: Pop
- Slim Jim Bites: Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA.

In [3]:
%%sql
SELECT
    t.genre_id genre_id,
    g.name genre,
    SUM(il.quantity) sold_units_abs,
    CAST(SUM(il.quantity) AS Float)/(
        SELECT 
            SUM(il.quantity) sold_total
        FROM genre g
        INNER JOIN track t ON t.genre_id=g.genre_id
        INNER JOIN invoice_line il ON il.track_id=t.track_id
        INNER JOIN invoice i ON i.invoice_id=il.invoice_id
        INNER JOIN customer c ON c.customer_id=i.customer_id
        WHERE c.country = "USA"
        ) sold_units_perc
FROM genre g
INNER JOIN track t ON t.genre_id=g.genre_id
INNER JOIN invoice_line il ON il.track_id=t.track_id
INNER JOIN invoice i ON i.invoice_id=il.invoice_id
INNER JOIN customer c ON c.customer_id=i.customer_id
WHERE c.country = "USA"
GROUP BY t.genre_id, g.name
ORDER BY sold_units_abs DESC;

Done.


genre_id,genre,sold_units_abs,sold_units_perc
1,Rock,561,0.5337773549000951
4,Alternative & Punk,130,0.12369172216936251
3,Metal,124,0.11798287345385347
14,R&B/Soul,53,0.05042816365366318
6,Blues,36,0.03425309229305423
23,Alternative,35,0.03330161750713606
7,Latin,22,0.02093244529019981
9,Pop,22,0.02093244529019981
17,Hip Hop/Rap,20,0.019029495718363463
2,Jazz,14,0.013320647002854425


According to these data, Red Tone and Slim Jim Bited will be successful, the third on is Meteor and the Girls but the difference with the fourth is not significative.

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [4]:
%%sql
SELECT
    e.first_name || " " || e.last_name employee_name,
    SUM(i.total) total_sales,
    e.title,
    e.hire_date
FROM customer c
INNER JOIN employee e ON e.employee_id=c.support_rep_id
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY employee_name;

Done.


employee_name,total_sales,title,hire_date
Jane Peacock,1731.5100000000039,Sales Support Agent,2017-04-01 00:00:00
Margaret Park,1584.0000000000034,Sales Support Agent,2017-05-03 00:00:00
Steve Johnson,1393.920000000002,Sales Support Agent,2017-10-17 00:00:00


The differences in terms of sales corresponds to the difference in hiring dates.

## Analyzing Sales by Country

The next task is to analyze the sales data for customers from each different country. I have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, I have to calculate data, for each country:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, I'll group these customers as "Other".

In [5]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 WHERE country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    c.country country,
    c.number_of_customers number_of_customers,
    c.value_of_sales value_of_sales,
    c.avg_sales_per_customer avg_sales_per_customer,
    c.avg_order_value avg_order_value
FROM (
    SELECT
        c.country country,
        COUNT(distinct c.customer_id) number_of_customers,
        SUM(c.unit_price) value_of_sales,
        SUM(c.unit_price) / count(distinct c.customer_id) avg_sales_per_customer,
        SUM(c.unit_price) / count(distinct c.invoice_id) avg_order_value,
        CASE
            WHEN c.country="Other" THEN 1
            ELSE 0
        END AS rank
    FROM country_or_other c
    GROUP BY country
    ORDER BY rank
    ) c
ORDER BY rank, value_of_sales DESC;

Done.


country,number_of_customers,value_of_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486


## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

I have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [6]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.

## Which artist is used in the most playlists?

In [7]:
%%sql
SELECT * from playlist_track

Done.


playlist_id,track_id
1,3402
1,3389
1,3390
1,3391
1,3392
1,3393
1,3394
1,3395
1,3396
1,3397


In [8]:
%%sql

SELECT 
    a.name singer_name, 
    count(a.name) number_of_playlist
FROM playlist_track pt
INNER JOIN track t on t.track_id = pt.track_id
INNER JOIN album ab on ab.album_id = t.album_id
INNER JOIN artist a on a.artist_id = ab.artist_id
GROUP BY singer_name
ORDER BY number_of_playlist DESC
LIMIT 10;
    

Done.


singer_name,number_of_playlist
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226
Lost,184
Pearl Jam,177
Eric Clapton,145
Faith No More,145
Lenny Kravitz,143


## How many tracks have been purchased vs not purchased?

In [9]:
%%sql

WITH total_tracks AS (
    SELECT COUNT(track_id) total FROM track
)

SELECT
    CAST(COUNT(distinct track_id)*100/(SELECT total FROM total_tracks) AS FLOAT) AS pct_purchased,
    100-CAST(COUNT(distinct track_id)*100/(SELECT total FROM total_tracks) AS FLOAT) AS pct_not_purchased
FROM invoice_line;

Done.


pct_purchased,pct_not_purchased
51.0,49.0


- 51% of the tracks have been purchased.
- 49% haven't.

In [10]:
%%sql
SELECT *
FROM media_type;

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


## Do protected vs non-protected media types have an effect on popularity?

In [11]:
%%sql
SELECT
    CASE
        WHEN mt.name LIKE "%Protected%" THEN "protected"
        ELSE "not_protected"
    END AS protection,
    COUNT(distinct il.track_id) selled_tracks 
FROM invoice_line il
INNER JOIN track t ON t.track_id=il.track_id
INNER JOIN media_type mt ON mt.media_type_id=t.media_type_id
GROUP BY protection;

Done.


protection,selled_tracks
not_protected,1652
protected,154
